In [0]:
import numpy as np
import pandas as pd
import pickle
import os
import gc

from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Input
from keras import backend as K
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD, Adam
import tensorflow as tf
import random

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
path = "gdrive/My Drive/NeuralNetCapstone/"
picklefile = "onechannel.pkl"

with open(os.path.join(path,picklefile), "rb") as f:
    images = pickle.load(f)

print ("Shape:",images.shape)

Shape: (51759, 128, 128, 1)


In [0]:
#Get y labels from data file
datasheetfile = "final_df.csv"
df = pd.read_csv(os.path.join(path,datasheetfile))

In [0]:
all_classes = df.columns.values[-14:]
y_true = df[all_classes].values

In [0]:
print(y_true.shape)

(51759, 14)


In [0]:
from PIL import Image

def get_extra_images(disease):
    path = "gdrive/My Drive/NeuralNetCapstone/"
    picklefile = None
    if disease == 0:
        picklefile = "gan_alec500.pkl"
    if disease == 4:
        picklefile = "gan_effusion.pkl"
    if disease == 7:
        picklefile = "hernia_aug.pkl"
    elif disease == 8:
        picklefile = "gan_infil.pkl"
    elif disease == 10:
        picklefile = "gan_nodule.pkl"
    elif disease == 12:
        picklefile = "pneumonia_aug.pkl"
    

    resized_images = []
    with open(os.path.join(path,picklefile), "rb") as f:
        new_images = pickle.load(f)
        print("Initial Shape: " + str(new_images.shape))

        if disease == 8 or disease == 0 or disease == 4 or disease == 10:
            for i in range(len(new_images)):
                im = (new_images[i]*255).astype('uint8').reshape([64, 64])
                #print(im.shape)
                im2 = Image.fromarray(im)
                im3 = im2.resize((128, 128), Image.BICUBIC)
                im4 = np.array(im3)
                resized_images.append(im4)
            new_images = resized_images
            del resized_images
            gc.collect()
    
    truth_array = []
    for i in range(len(new_images)):
        truth_vals = [0]*14
        truth_vals[disease] = 1
        truth_array.append(truth_vals)
    truth_array = np.array(truth_array)
    
    #print("New Images Shape: " + str(new_images.shape))
    #print("Ground Truth Shape: " + str(truth_array.shape))
    #print(truth_array[0:10])
    
    if disease == 8 or disease == 0 or disease == 4 or disease == 10:
        new_images = np.array(new_images).reshape([-1, 128, 128, 1])
    
    return np.array(new_images[0:3000]), np.array(truth_array[0:3000])
  
aug_x, aug_y = get_extra_images(10)
print("Augmented X Shape: " + str(aug_x.shape))

Initial Shape: (7386, 64, 64, 1)
Augmented X Shape: (3000, 128, 128, 1)


In [0]:
#aug_x.reshape([-1, 128, 128, 1])
aug_x.shape

(3000, 128, 128, 1)

In [0]:
train_X, test_X, train_y, test_y = train_test_split(images, y_true, test_size=0.2, random_state=0)
train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, test_size=0.2, random_state=0)

del images
gc.collect()

20

In [0]:

train_X.shape

(33125, 128, 128, 1)

In [0]:
train_X = np.vstack((train_X, aug_x))
train_y = np.vstack((train_y, aug_y))
print(train_X.shape)
print(train_y.shape)

(36125, 128, 128, 1)
(36125, 14)


In [0]:
del aug_x
del aug_y
gc.collect()

0

In [0]:
def get_binary_truth(y):
    binary_y = []
    for i in range(14):
        class_y = []
        for j in range(len(y)):
            val = int(y[j][i])
            one_hot = np.array([0, 0])
            one_hot[val] = 1
            class_y.append(one_hot)
        class_y = np.array(class_y)
        class_y = np.reshape(class_y, (-1, 2))
        binary_y.append(np.array(class_y))
    return binary_y

train_binary_y = get_binary_truth(train_y)
val_binary_y = get_binary_truth(val_y)
test_binary_y = get_binary_truth(test_y)

print(np.shape(train_binary_y))
print(train_binary_y[13])
print(train_binary_y[13].shape)

(14, 36125, 2)
[[1 0]
 [1 0]
 [1 0]
 ...
 [1 0]
 [1 0]
 [1 0]]
(36125, 2)


#Functions to create CNN

In [0]:
act_functions = {'relu':tf.nn.relu, 'sigmoid':tf.nn.sigmoid, 'tanh':tf.nn.tanh}

def conv2d(x, W, b, function, strides=1):
    
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    
    func = act_functions[function]
    
    return func(x)
    
def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

    
def getWeightsAndBiases(size, filters, outputs, pooling, channels, neural_neurons, classes, beta):
    weights = {}
    biases = {}
    regularizer = tf.contrib.layers.l2_regularizer(scale=beta)
    
    #Generate weights and biases for convolutional layers
    for i in range(len(outputs)):
        filter_size = filters[i]
        #Size of inputs to layer is the same as outputs of previous layer
        inputs = channels
        if i > 0:
            inputs = outputs[i-1]
    
        #Create weights and biases dictionary
        weight_shape = (filter_size, filter_size, inputs, outputs[i])
        bias_shape = (outputs[i])
        
        #Update size of images in layers based on pooling filter
        size = int(np.ceil(size/pooling[i]))
        
        weights[str(i)] = tf.get_variable("W" + str(i), 
                                          shape = weight_shape, 
                                          initializer = tf.contrib.layers.xavier_initializer(),
                                          regularizer = regularizer)
        biases[str(i)] = tf.get_variable("B" + str(i), 
                                         shape = (bias_shape), 
                                         initializer = tf.contrib.layers.xavier_initializer())
        
    #Generate weights and biases for neural network layers
    weights[str(i+1)] = tf.get_variable("W" + str(i+1), 
                                        shape = (size*size*outputs[i], neural_neurons), 
                                        initializer = tf.contrib.layers.xavier_initializer(),
                                        regularizer = regularizer)
    biases[str(i+1)] = tf.get_variable("B" + str(i+1),
                                        shape = (neural_neurons),
                                        initializer = tf.contrib.layers.xavier_initializer())
    
    #Generate weights and biases for output layer
    weights[str(i+2)] = tf.get_variable("W" + str(i+2),
                                        shape = (neural_neurons, classes),
                                        initializer = tf.contrib.layers.xavier_initializer(),
                                        regularizer = regularizer)
    biases[str(i+2)] = tf.get_variable("B" + str(i+2),
                                        shape = (classes),
                                        initializer = tf.contrib.layers.xavier_initializer())
    
    return weights, biases
    
def createCNN(x, weights, biases, pooling, dropout, function):
    
    prev_layer = x
    print("Input Layer Shape: " + str(prev_layer.shape))
    for i in range(len(pooling)):
        #Create layer
        layer = conv2d(prev_layer, weights[str(i)], biases[str(i)], function)
        #Downsample layer
        layer = maxpool2d(layer, k=pooling[i])
        
        prev_layer= layer
    
    #Add dropout layer
    drop1_layer = tf.layers.dropout(inputs = prev_layer, rate=dropout[0])
    
    #Flatten CNN
    fc1 = tf.reshape(drop1_layer, [-1, weights[str(i+1)].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights[str(i+1)]), biases[str(i+1)])
    func = act_functions[function]
    fc1 = func(fc1)
    
    #Add dropout layer
    dropout_layer = tf.layers.dropout(inputs = fc1, rate=dropout[1])
    
    #Perform final layer operations and output appropriate number of outputs
    out = tf.add(tf.matmul(dropout_layer, weights[str(i+2)]), biases[str(i+2)])
    return out

#Create and Train CNN

In [0]:
#Reset TensorFlow
tf.reset_default_graph()
preds = []
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

#Create data placeholders for inputs and input labels (None value is batch size)
size = train_X.shape[1]
classes = 2
channels = train_X.shape[3]
x = tf.placeholder('float', [None, size, size, channels])
y = tf.placeholder('float', [None, classes])

#Hyperparameters
outputs = [32, 64]
learn_rate = 0.000001
dropout_rate = [0.1, 0.5]
batch_size = 256
neural_neurons = 1024
curr_class = 10
beta = 0.05

filters = [3, 4]
pooling = [2, 2]
func_name = 'relu'
weights, biases = getWeightsAndBiases(size, filters, outputs, pooling, channels, neural_neurons, classes, beta)
predictor = createCNN(x, weights, biases, pooling, dropout_rate, func_name)

#Calculate weight for cost function
weight_ratio = 0 
current = train_binary_y[curr_class]
for j in range(len(current)):
  weight_ratio += current[j][1]
weight_ratio /= len(current)
print("Weight Ratio: " + str(weight_ratio))

#Populate and train CNN
#cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictor, labels=y))
cost = tf.reduce_mean(tf.nn.weighted_cross_entropy_with_logits(logits=predictor, targets=y, pos_weight = 1/0.122))
optimizer = tf.train.AdamOptimizer(learning_rate= learn_rate).minimize(cost)

#Create nodes to assess network accuracy
correct_prediction = tf.equal(tf.argmax(predictor, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#Initialize all variables (including weights and biases)
init = tf.global_variables_initializer()

#Run CNN
with tf.Session() as sess:
    sess.run(init)
    train_loss = []
    val_loss = []
    train_accuracy = []
    val_accuracy = []
    summary_writer = tf.summary.FileWriter('./Output', sess.graph)

    epochs = 30
    batch_size = batch_size

    key = str(outputs)+"_"+str(learn_rate)+"_"+str(dropout_rate)+"_"+str(batch_size)
    print(key)
    
    train_indices = range(0, len(train_X))
    train_indices = shuffle(train_indices)

    val_indices = range(0, len(val_X))
    val_indices = shuffle(val_indices)
    
    test_indices = range(0, len(test_X))
    test_indices = shuffle(test_indices)
    
    #Iterate for certain number of epochs
    for i in range(epochs):
        #Iterate through all batches in single epoch
        for batch in range(len(train_X)//batch_size):
            batch_x = generate_batch(train_X, batch, batch_size, train_indices)
            batch_y = generate_batch(train_binary_y[curr_class], batch, batch_size, train_indices)
            #batch_x = train_X[batch*batch_size:min((batch+1)*batch_size,len(train_X))]
            #batch_y = train_binary_y[curr_class][batch*batch_size:min((batch+1)*batch_size,len(train_binary_y[curr_class]))]

            #Run optimizer
            opt = sess.run(optimizer, feed_dict = {x: batch_x, y: batch_y})
            loss, acc = sess.run([cost, accuracy], feed_dict = {x: batch_x, y: batch_y})

        print("Epoch " + str(i) + ", Loss= " + "{:.6f}".format(loss) + ", Training Accuracy= " + "{:.5f}".format(acc))

        #Calculate validation set statistics
        tps = fps = tns = fns = 0
        valid_accs = []
        valid_losses = []
        valid_aucs = []
        val_batch_size = 2048
        for val_batch in range(len(val_X)//val_batch_size):
            val_batch_x = generate_batch(val_X, val_batch, val_batch_size, val_indices)
            val_batch_y = generate_batch(val_binary_y[curr_class], val_batch, val_batch_size, val_indices)
            #val_batch_x = val_X[val_batch*val_batch_size:min((val_batch+1)*val_batch_size,len(val_X))]
            #val_batch_y = val_binary_y[curr_class][val_batch*val_batch_size:min((val_batch+1)*val_batch_size,len(val_binary_y[curr_class]))]
            preds = predictor.eval(feed_dict = {x: val_batch_x})
            valid_acc, valid_loss = sess.run([accuracy, cost], feed_dict = {x: val_batch_x, y: val_batch_y})
            valid_accs.append(valid_acc)
            valid_losses.append(valid_loss)
            tp, fp, tn, fn, auc = conf_matrix(preds, val_batch_y)
            tps += tp
            fps += fp
            tns += tn
            fns += fn
            valid_aucs.append(auc)
        acc = (tps+tns)/(tps+fps+tns+fns)
        recall = tps/(tps+fns)
        precision = 0
        if (tps+fps) > 0:
            precision = tps/(tps+fps)
        f1 = 2*tps/(2*tps+fps+fns)

        print("Validation Accuracy:","{:.5f}".format(np.mean(valid_accs)))
        print("Validation Loss:","{:.5f}".format(np.mean(valid_losses)))
        print("Accuracy: " + str(acc))
        print("Recall: " + str(recall))
        print("Precision: " + str(precision))
        print("F1: " + str(f1))
        print("AUC: " + str(np.mean(valid_aucs)))
        print("")
        
        #Calculate testing set statistics
        tps = fps = tns = fns = 0
        test_accs = []
        test_losses = []
        test_aucs = []
        test_batch_size = val_batch_size
        for test_batch in range(len(test_X)//test_batch_size):
            test_batch_x = generate_batch(test_X, test_batch, test_batch_size, test_indices)
            test_batch_y = generate_batch(test_binary_y[curr_class], test_batch, test_batch_size, test_indices)
            #test_batch_x = test_X[test_batch*test_batch_size:min((test_batch+1)*test_batch_size,len(test_X))]
            #test_batch_y = test_binary_y[curr_class][test_batch*test_batch_size:min((test_batch+1)*test_batch_size,len(test_binary_y[curr_class]))]
            preds = predictor.eval(feed_dict = {x: test_batch_x})
            test_acc, test_loss = sess.run([accuracy, cost], feed_dict = {x: test_batch_x, y: test_batch_y})
            test_accs.append(test_acc)
            test_losses.append(test_loss)
            tp, fp, tn, fn, auc = conf_matrix(preds, test_batch_y)
            tps += tp
            fps += fp
            tns += tn
            fns += fn
            test_aucs.append(auc)
        acc = (tps+tns)/(tps+fps+tns+fns)
        recall = tps/(tps+fns)
        precision = 0
        if (tps+fps) > 0:
            precision = tps/(tps+fps)
        f1 = 2*tps/(2*tps+fps+fns)

        print("Testing Accuracy:","{:.5f}".format(np.mean(test_accs)))
        print("Testing Loss:","{:.5f}".format(np.mean(test_losses)))
        print("Accuracy: " + str(acc))
        print("Recall: " + str(recall))
        print("Precision: " + str(precision))
        print("F1: " + str(f1))
        print("AUC: " + str(np.mean(test_aucs)))
        print("")
        
        '''valid_size = 2000
        preds = predictor.eval(feed_dict = {x: val_X[0:valid_size]})
        valid_acc, valid_loss = sess.run([accuracy, cost], feed_dict = {x: val_X[0:valid_size], y: val_binary_y[curr_class][0:valid_size]})
        train_loss.append(loss)
        val_loss.append(valid_loss)
        train_accuracy.append(acc)
        val_accuracy.append(valid_acc)
        print("Validation Accuracy:","{:.5f}".format(valid_acc))
        print("Validation Loss:","{:.5f}".format(valid_loss))
        conf_matrix(preds, val_binary_y[curr_class], print_stats = True)'''

        '''#Calculate accuracy for test set
        test_acc, test_loss = sess.run([accuracy, cost], feed_dict = {x: test_X, y: test_y})
        print("Testing Accuracy:","{:.5f}".format(test_acc))
        print("Testing Loss:","{:.5f}".format(test_loss))'''

Input Layer Shape: (?, 128, 128, 1)
Weight Ratio: 0.19587543252595155
[32, 64]_1e-06_[0.1, 0.5]_256
Epoch 0, Loss= 1.504340, Training Accuracy= 0.82031
Validation Accuracy: 0.82776
Validation Loss: 1.51983
Accuracy: 0.8277587890625
Recall: 0.075
Precision: 0.13368983957219252
F1: 0.09609224855861627
AUC: 0.5218929970323418

Testing Accuracy: 0.83643
Testing Loss: 1.41674
Accuracy: 0.83642578125
Recall: 0.08799342105263158
Precision: 0.1589895988112927
F1: 0.11328745367919534
AUC: 0.543435785558849

Epoch 1, Loss= 1.340096, Training Accuracy= 0.82422
Validation Accuracy: 0.84241
Validation Loss: 1.36008
Accuracy: 0.8424072265625
Recall: 0.062
Precision: 0.1493975903614458
F1: 0.08763250883392226
AUC: 0.5318641249962625

Testing Accuracy: 0.84922
Testing Loss: 1.27439
Accuracy: 0.84921875
Recall: 0.0649671052631579
Precision: 0.16255144032921812
F1: 0.09283196239717979
AUC: 0.5503546098159726

Epoch 2, Loss= 1.339429, Training Accuracy= 0.85156
Validation Accuracy: 0.86560
Validation Los

KeyboardInterrupt: ignored

In [0]:
from sklearn.utils import shuffle

def generate_batch(data, batch, batch_size, shuffled_indices):
    size = batch_size
    if size+(batch)*batch_size > len(data):
        size = len(data) - batch*batch_size
    data_out = []
    for i in range(size):
        index = shuffled_indices[batch*batch_size+i]
        val = data[index]
        data_out.append(val)
        
    return np.array(data_out)

In [0]:
from scipy.stats import logistic
from sklearn.metrics import roc_auc_score

def conf_matrix(preds, truth_list, print_stats = False):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    
    #print(preds[0:10])
    #print("Sigmoid: ")
    #print(logistic.cdf(preds[0:10]))
    #print(truth_list.shape)
    #print(preds.shape)
    
    auc = roc_auc_score(truth_list, logistic.cdf(preds))
    #print("AUC: " + str(auc))

    for i in range(len(preds)):
      pred = [1, 0]
      if preds[i][1] > preds[i][0]:
        pred = [0, 1]
      preds[i] = pred

    for i in range(len(preds)):
      pred = preds[i][1]
      truth = truth_list[i][1]
      if pred == 1 and truth == 1:
        tp += 1
      elif pred == 1 and truth == 0:
        fp += 1
      elif pred == 0 and truth == 0:
        tn += 1
      elif pred == 0 and truth == 1:
        fn += 1

    #print("True Positive: " + str(tp))
    #print("False Positive: " + str(fp))
    #print("True Negative: " + str(tn))
    #print("False negative: " + str(fn))

    accuracy = (tp+tn)/(tp+fp+tn+fn)
    recall = tp/(tp+fn)
    precision = 0
    if (tp+fp) > 0:
        precision = tp/(tp+fp)
    f1 = 2*tp/(2*tp+fp+fn)

    if print_stats == True:
        print("Accuracy: " + str(accuracy))
        print("Recall: " + str(recall))
        print("Precision: " + str(precision))
        print("F1: " + str(f1))
        print("")
    
    return tp, fp, tn, fn, auc